# BMI/CS 576 Fall 2019 - HW3
The objectives of this homework are to practice phylogenetic tree reconstruction algorithms.  Specifically, you will gain practice with the following techniques:

* Neighbor-joining
* branch and bound
* unweighted and weighted parsimony

## HW policies
Before starting this homework, please read over the [homework policies](https://canvas.wisc.edu/courses/167969/pages/hw-policies) for this course.  In particular, note that homeworks are to be completed *individually*.

You are welcome to use any code from the weekly notebooks in your solutions to the HW.

## PROBLEM 1: Neighbor-joining (20 points)

Construct a tree from these distances using the neighbor-joining algorithm. Show your updated distance matrix after each merge and give branch lengths for the final tree.

|       | A | B | C | D | E |
|-------|---|---|---|---|---|
| **A** |   | 7 | 13|  9| 10| 
| **B** |   |   |  8|  4|  5|
| **C** |   |   |   |  6|  9|
| **D** |   |   |   |   |  5|
| **E** |   |   |   |   |  &nbsp; |


### BEGIN SOLUTION template=Your solution to Problem 1 here
*Note that in steps 1, 2, and 3, there are ties for the minimum $D_{ij}$, however, all choices for the next pair of nodes to join lead to the same tree.*

![neighbor_joining_solution](neighbor_joining.png)

### END SOLUTION

## Trees for Problem 2 and 3
![parsimony_trees](parsimony_trees.png)

## PROBLEM 2: Unweighted parsimony (20 POINTS)

Suppose we are given five DNA sequences (1, 2, 3, 4, and 5), each of which is one base long. The figure above gives two possible trees relating these five sequences.

**(a)** For each of the two trees, find the minimal cost of the tree using Fitch’s algorithm (unweighted parsimony). Show the intermediate computations of the algorithm.

**(b)** For each of the two trees, determine an assignment of ancestral bases that achieves the minimal cost that you found in (a).

**(c)** Which tree would be preferred when using unweighted parsimony?

### BEGIN SOLUTION template=Your solution to Problem 2 here

![problem2_solution](parsimony_trees_unweighted_solution.png)
In the figure above, each internal node, $i$, is labeled by the set of characters, $R_i$, that minimize the cost of the subtree rooted by that node. The bold characters are one assignment of ancestral bases that achieve the minimal cost. For both trees, there is one alternative assignment of ancestral bases that differs only in that it has a T at the root node instead of a G.

**(a)** The minimal cost of both trees is 2.  See the figure above for the intermediate calculations.

**(b)** See the figure above for the assignment of ancestral bases that achieves the minimal cost of 2.

**(c)** Since both trees have the same cost, neither would be preferred over the other.
### END SOLUTION

## PROBLEM 3: Weighted parsimony (20 POINTS)

(25points) Suppose we are given the same five sequences as in Problem 2 and the same two possible trees. Given the weighted parsimony costs given in the matrix below:

![weighted_parsimony_weights](weighted_parsimony_weights.png)

**(a)** For each of the two trees, find the minimal cost of the tree using the weighted parsimony algorithm. Show the intermediate computations of the algorithm.

**(b)** For each of the two trees, determine an assignment of ancestral bases that achieves the minimal cost that you found in (a).

**(c)** Which tree would be preferred when using weighted parsimony with the given costs?


### BEGIN SOLUTION template=Your solution to Problem 3 here
![problem3_solution](parsimony_trees_weighted_solution.png)
In the figure above, for each internal node, $i$, given is the minimum cost of the subtree rooted at $i$, $R_i(c)$, for each character $c$. The bold characters labeling the $R$ vectors indicate the an assignment of ancestral bases that achieve the minimum cost. For both trees, there is one alternative assignment of ancestral bases that differs only in that it has a T at the root node instead of a G.

**(a)** The minimal cost of Tree 1 is 4. The minimal cost of Tree 2 is 3.  See the figure above for the intermediate calculations.

**(b)** See the figure above for the assignment of ancestral bases that achieves the minimal cost for each tree.

**(c)** Tree 2 would be preferred because it has the smaller cost (3).

### END SOLUTION

## PROBLEM 4: Branch and bound (40 POINTS)

Implement the basic branch and bound algorithm (page 7 of the "Searching through tree space" lecture slides) for finding an optimal tree with an unweighted parsimony objective function.  Your implementation should be broken down into three functions, `best_tree_branch_and_bound`, `branch`, and `bound`, which you are to implement below.

Some implementation specifications for each function:
### `best_tree_branch_and_bound`

* You should use the [Heap queue algorithm](https://docs.python.org/3/library/heapq.html) module functions to efficiently maintain the queue (Q) 
* Q should be sorted by lower bound of the trees, with ties broken by lexicographical ordering of the trees' newick strings.
* The algorithm should begin with an unrooted tree consisting of the first three names in `sequence_names`, and add leaves in the order in which they appear in the sequence_names list.

### `branch`

* This function should call the `add_leaf` function (provided) to grow an unrooted tree with the next leaf in all possible ways

### `bound`

* You will likely want to take advantage of your work in the day 16 notebook for this function
* To convert an unrooted tree to a rooted tree (such that you can use your parsimony scoring code), it is recommended that you call the provided `root` function

In [1]:
# Code for PROBLEM 4
# You are welcome to develop your code as a separate Python module
# and import it here if that is more convenient for you.

import heapq
import toytree

def best_tree_branch_and_bound(alignment, sequence_names, branch, bound):
    """Computes an optimal (lowest scoring) tree using a branch and bound algorithm.
    
    Args:
        alignment: a list of strings corresponding to the rows of a multiple alignment.
        sequence_names: a list of the names of the sequences in the same order as the
                        rows of the multiple alignment.
        branch: a function that grows a partial tree in multiple ways
        bound: a function that computes the lower bound of a partial tree
    Returns:
        A tuple (score, newick_string) where newick_string is a Newick formatted string
        representing the optimal tree (unrooted) and score is its score.
    """
    ### BEGIN SOLUTION
    # make a tree with the first three names
    first_three_tree_newick = "({},{},{});".format(*sequence_names[0:3])
    
    # initialize a queue with the first tree and its lower bound
    first_tree_bound = bound(first_three_tree_newick, alignment, sequence_names)
    q = [(first_tree_bound, first_three_tree_newick)]
    
    while True:
        t_bound, t_newick_tree = heapq.heappop(q)
        t_tree = toytree.tree(t_newick_tree)
        if t_tree.ntips == len(sequence_names):
            return t_bound, t_newick_tree
        else:
            for new_tree in branch(t_newick_tree, sequence_names[t_tree.ntips]):
                new_tree_bound = bound(new_tree, alignment, sequence_names)
                heapq.heappush(q, (new_tree_bound, new_tree))
    ### END SOLUTION
    
def branch(newick_tree, next_leaf_name):
    """Grows a partial unrooted tree by adding the next leaf in all possible ways.
    
    Args:
        newick_tree: a partial unrooted tree as a Newick string
        next_leaf_name: the name of the next leaf to add to the tree
    Returns:
        A list of Newick strings representing all possible ways in which to add the next leaf.
    """
    ### BEGIN SOLUTION    
    tree = toytree.tree(newick_tree)
    num_edges = 2 * tree.ntips - 3
    return [add_leaf(newick_tree, next_leaf_name, i) for i in range(1, num_edges + 1)]
    ### END SOLUTION

def bound(newick_tree, alignment, sequence_names):
    """Computes a lower bound for the unweighted parsimony score of a full tree that can be 
    grown from a given partial tree.
    
    Args:
        newick_tree: a partial unrooted tree as a Newick string
        alignment: a list of strings corresponding to the rows of a multiple alignment.
        sequence_names: a list of the names of the sequences in the same order as the
                        rows of the multiple alignment.
    Returns:
        The lower bound as an integer.
    """
    ### BEGIN SOLUTION    
    rooted_newick = root(newick_tree)
    return score_tree_parsimony(toytree.tree(rooted_newick), alignment, sequence_names)
    ### END SOLUTION

def add_leaf(newick_tree, leaf_name, edge_index):
    """Adds a new leaf on to the specified edge in an unrooted tree.
    
    Args:
        newick_tree: a partial unrooted tree as a Newick string
        leaf_name: the name of the next leaf to add to the tree
        edge_index: the index (from 1 to the number of edges) of the edge on
                    which to add the leaf.  Edges are ordered by the order
                    in which their child node is encountered in a preorder
                    traversal of the tree.
    Returns:
        A newick string representing the tree with the added leaf.
    """
    new_tree = toytree.tree(newick_tree)
    if len(new_tree.treenode.children) != 3:
        raise ValueError("Tree does not look unrooted: " +  newick_tree)
    for i, node in enumerate(new_tree.treenode.traverse("preorder")):
        if i == edge_index:
            break
    parent = node.up
    node.detach()
    new_internal_node = parent.add_child()
    new_internal_node.add_child(node)
    new_internal_node.add_child(name=leaf_name)
    return new_tree.treenode.write(format=9)

def root(newick_tree):
    """Converts an unrooted tree into a rooted tree.
    (useful for scoring an unrooted tree with parsimony).
    
    Args:
        newick_tree: an unrooted tree as a Newick string
    Returns:
        A rooted version of the tree as a newick string.
    """
    unrooted_tree = toytree.tree(newick_tree)
    if len(unrooted_tree.treenode.children) != 3:
        raise ValueError("Tree does not look unrooted: " +  newick_tree)
    unrooted_tree_root = unrooted_tree.treenode
    first_child = unrooted_tree.treenode.children[0]
    first_child.detach()
    rooted_tree_root = toytree.TreeNode.TreeNode()
    rooted_tree_root.add_child(first_child)
    rooted_tree_root.add_child(unrooted_tree_root)
    return rooted_tree_root.write(format=9)

### BEGIN SOLUTION template=
def fitch_score(tree, leaf_states):
    """Runs the first stage of Fitch's algorithm for
       the given tree and character states as the leaves.
    
    Args:
        tree: a toytree tree.
        leaf_states: a dictionary mapping leaf names to characters.  
    Returns:
        The minimum cost of the tree (minimum number of changes required to explain
        the leaf data)
    """
    R = {}
    num_changes = 0
    for node in tree.treenode.traverse("postorder"):
        if node.is_leaf():
            R[node.name] = {leaf_states[node.name]}
        else:
            left_states, right_states = [R[child.name] for child in node.children]
            intersection = left_states & right_states
            if intersection:
                R[node.name] = intersection
            else:
                R[node.name] = left_states | right_states
                num_changes += 1
    return num_changes

def score_tree_parsimony(tree, alignment, sequence_names):
    """Computes the parsimony score for a given tree and alignment.
    
    Args:
        tree: a toytree tree object
        alignment: a list of strings corresponding to the rows of a multiple alignment.
        sequence_names: a list of the names of the sequences in the same order as the
                        rows of the multiple alignment.
    Returns:
        The parsimony score (a number)
    """
    return sum(fitch_score(tree, leaf_states)
               for leaf_states in alignment_leaf_states_list(alignment, sequence_names))
    
def alignment_leaf_states_list(alignment, sequence_names):
    """Returns a list of dictionaries, where each dictionary corresponds to the leaf states
    for a column of the alignment."""
    return [dict(zip(sequence_names, column)) for column in zip(*alignment)]
### END SOLUTION

## Tests for PROBLEM 4

### Data sets for testing

In [2]:
import fasta
def read_names_and_alignments_from_fasta(filename):
    return zip(*fasta.read_sequences_from_fasta_file(filename))

v3_sequence_names, v3_alignment = read_names_and_alignments_from_fasta("v3_alignment.fasta")

v3_big_sequence_names, v3_big_alignment = read_names_and_alignments_from_fasta("v3_big_alignment.fasta")

medium_num_seqs = 7
v3_medium_alignment = v3_big_alignment[:medium_num_seqs]
v3_medium_sequence_names = v3_big_sequence_names[:medium_num_seqs]

### Tests

In [3]:
# tests for branch3
assert sorted(branch('(D,PA,PB);', 'C1')) == ['(D,PA,(PB,C1));', 
                                              '(D,PB,(PA,C1));', 
                                              '(PA,PB,(D,C1));']
print("SUCCESS: branch passed all tests")

SUCCESS: branch passed all tests


In [4]:
# tests for branch4
assert sorted(branch('(D,PA,(PB,C1));', 'C2')) == ['(D,(PB,C1),(PA,C2));',
                                                   '(D,PA,((PB,C1),C2));',
                                                   '(D,PA,(C1,(PB,C2)));',
                                                   '(D,PA,(PB,(C1,C2)));',
                                                   '(PA,(PB,C1),(D,C2));']
print("SUCCESS: branch4 passed all tests")

SUCCESS: branch4 passed all tests


In [5]:
# tests for branch5
assert sorted(branch('(D,(PB,C1),(PA,C2));', 'C3')) == ['((PB,C1),(PA,C2),(D,C3));',
                                                        '(D,(C1,(PB,C3)),(PA,C2));',
                                                        '(D,(PA,C2),((PB,C1),C3));',
                                                        '(D,(PB,(C1,C3)),(PA,C2));',
                                                        '(D,(PB,C1),((PA,C2),C3));',
                                                        '(D,(PB,C1),(C2,(PA,C3)));',
                                                        '(D,(PB,C1),(PA,(C2,C3)));']
print("SUCCESS: branch5 passed all tests")

SUCCESS: branch5 passed all tests


In [6]:
# tests for bound_one_column
v3_alignment_column0 = [s[0] for s in v3_alignment]
v3_alignment_column3 = [s[3] for s in v3_alignment]
v3_alignment_column29 = [s[29] for s in v3_alignment]
assert bound('(D,PA,PB);', v3_alignment_column0, v3_sequence_names) == 0
assert bound('(D,PA,PB);', v3_alignment_column3, v3_sequence_names) == 1
assert bound('((C1,C2),(PA,PB),D);', v3_alignment_column29, v3_sequence_names) == 1
print("SUCCESS: bound_one_column passed all tests")

SUCCESS: bound_one_column passed all tests


In [7]:
# tests for bound_alignment
assert bound('(D,PA,PB);', v3_alignment, v3_sequence_names) == 25
assert bound('((D, C1),PA,PB);', v3_alignment, v3_sequence_names) == 48
assert bound('((C1,PA),(C2,PB),D);', v3_alignment, v3_sequence_names) == 64
print("SUCCESS: bound_alignment passed all tests")

SUCCESS: bound_alignment passed all tests


In [8]:
class CallCounter:
    def __init__(self, func):
        self._func = func
        self._num_calls = 0
    def __call__(self, *args, **kwds):
        self._num_calls += 1
        return self._func(*args, **kwds)
    def num_calls(self):
        return self._num_calls
    def reset(self):
        self._num_calls = 0

In [9]:
# tests for branch_and_bound_v3
counting_bound = CallCounter(bound)
assert best_tree_branch_and_bound(v3_alignment, v3_sequence_names, branch, counting_bound) == (58, '(D,PA,(PB,(C1,C2)));')
assert counting_bound.num_calls() == 19
print("SUCCESS: branch_and_bound_v3 passed all tests")

SUCCESS: branch_and_bound_v3 passed all tests


In [10]:
# tests for branch_and_bound_v3_medium
counting_bound = CallCounter(bound)
assert best_tree_branch_and_bound(v3_medium_alignment, v3_medium_sequence_names, branch, counting_bound) == (75, '(C09,(C35,(PD1,PD3)),(PA5,(PB6,PB8)));')
assert counting_bound.num_calls() == 133
print("SUCCESS: branch_and_bound_v3 passed all tests")

SUCCESS: branch_and_bound_v3 passed all tests


In [11]:
# tests for branch_and_bound_v3_big
### BEGIN HIDDEN TESTS
counting_bound = CallCounter(bound)
assert best_tree_branch_and_bound(v3_big_alignment, v3_big_sequence_names, branch, counting_bound) == (99, '(C09,((PB6,PB8),(PA5,D1)),((PF1,PF3),(C35,(PD1,PD3))));')
assert counting_bound.num_calls() == 2998
print("SUCCESS: branch_and_bound_v3 passed all tests")
### END HIDDEN TESTS

SUCCESS: branch_and_bound_v3 passed all tests
